# Training Data Generation

In [1]:
import os
import pandas as pd

from tqdm import tqdm
from constants import *

Define main procedures

In [2]:
def prepare_orbit(orbit_id):
    """Load the specified orbit and preprocess it for merging."""

    file = os.path.join(ORBIT_DIR, ORBIT_FILE(orbit_id))
    df_stats.loc[orbit_id, "does_not_exist"] = not os.path.exists(file)

    if not df_stats.loc[orbit_id, "does_not_exist"]:
        df_orbit = pd.read_csv(file, index_col=DATE_COL, parse_dates=True)

        # determine validity criteria for orbit
        df_stats.loc[orbit_id, "total_nan_count"] = (
                df_orbit.isnull().sum().sum()
        )
        df_stats.loc[orbit_id, "missing_entry_count"] = (
                (df_orbit.index[-1] - df_orbit.index[0]).total_seconds() + 1 - len(df_orbit)
        )
        df_stats.loc[orbit_id, "has_outside_labels"] = (
                df_labels.loc[orbit_id, "SK outer in"] < df_orbit.index[0]
                or df_orbit.index[-1] < df_labels.loc[orbit_id, "SK outer out"]
        )
        
        if df_stats.loc[orbit_id].sum() == 0:
            df_orbit[ORBIT_COL] = orbit_id  # add orbit id
            return df_orbit
        else:
            return None  # rule out invalid orbit

In [3]:
def assign_labels(df_train, df_labels):
    """Assign labels to the training time series using event boundary labels."""

    # interplanetary magnetic field
    df_train[LABEL_COL] = 0

    for row in tqdm(range(len(df_labels))):
        event = lambda col: df_labels.iloc[row][col]

        # bow shock crossings
        df_train.loc[event("SK outer in"):event("SK inner in"), LABEL_COL] = 1
        df_train.loc[event("SK inner out"):event("SK outer out"), LABEL_COL] = 1

        # magnetosheath
        df_train.loc[event("SK inner in"):event("MP outer in"), LABEL_COL] = 2
        df_train.loc[event("MP outer out"):event("SK inner out"), LABEL_COL] = 2

        # magnetopause crossings
        df_train.loc[event("MP outer in"):event("MP inner in"), LABEL_COL] = 3
        df_train.loc[event("MP inner out"):event("MP outer out"), LABEL_COL] = 3

        # magnetosphere
        df_train.loc[event("MP inner in"):event("MP inner out"), LABEL_COL] = 4

Load label descriptor file

In [4]:
df_labels = pd.read_csv(os.path.join(LABEL_DIR, LABEL_FILE),
                        index_col=ORBIT_COL,
                        parse_dates=list(range(1, 9)))
df_labels = df_labels.dropna()  # disregard incompletely labeled orbits
print(f"#healthy orbits: {len(df_labels)}")

#healthy orbits: 4019


Combine orbit data into one big frame

In [5]:
df_stats = pd.DataFrame(
    index=df_labels.index,
    columns=["does_not_exist",
             "total_nan_count",
             "missing_entry_count",
             "has_outside_labels"]
)

df_train = pd.concat(map(prepare_orbit, tqdm(df_labels.index)))
df_train.iloc[43462:43467]

100%|██████████| 4019/4019 [14:23<00:00,  4.66it/s]


,X_MSO,Y_MSO,Z_MSO,BX_MSO,BY_MSO,BZ_MSO,DBX_MSO,DBY_MSO,DBZ_MSO,RHO_DIPOLE,...,Y,Z,VX,VY,VZ,VABS,D,COSALPHA,EXTREMA,ORBIT
DATE,,,,,,,,,,,,,,,,,,,,,
2011-03-24 19:46:22,6897.941,5403.502,-15374.343,29.343,-2.064,-9.552,0.299,1.637,1.395,18118.125361,...,3.802753e+07,5.912986e+06,-47.808573,-28.603154,2.049779,55.749454,4.915513e+07,0.670984,0,2
2011-03-24 19:46:23,6898.235,5403.903,-15374.070,29.540,-1.713,-8.633,0.809,1.914,2.789,18118.117945,...,3.802753e+07,5.912986e+06,-47.808573,-28.603154,2.049796,55.749455,4.915513e+07,0.670984,2,2
2011-03-24 19:46:23,6898.235,5403.903,-15374.070,29.540,-1.713,-8.633,0.809,1.914,2.789,18118.117945,...,3.802753e+07,5.912986e+06,-47.808573,-28.603154,2.049796,55.749455,4.915513e+07,0.670984,2,3
2011-03-24 19:46:24,6898.530,5404.304,-15373.797,29.849,-0.977,-10.833,1.065,4.469,3.514,18118.110927,...,3.802753e+07,5.912986e+06,-47.808573,-28.603154,2.049746,55.749453,4.915513e+07,0.670984,0,3
2011-03-24 19:46:25,6898.825,5404.705,-15373.523,29.333,-2.752,-12.014,1.250,5.516,5.414,18118.103052,...,3.802753e+07,5.912986e+06,-47.808573,-28.603154,2.049724,55.749452,4.915513e+07,0.670984,0,3


Save orbit validity statistics

In [7]:
df_stats.to_excel(os.path.join(MERGED_DIR, "orbit_statistics.xlsx"))

Assign labels to the instances of the training data

In [11]:
assign_labels(df_train, df_labels)
df_train.iloc[16605:16610]

100%|██████████| 4019/4019 [5:55:17<00:00,  5.30s/it]  


,X_MSO,Y_MSO,Z_MSO,BX_MSO,BY_MSO,BZ_MSO,DBX_MSO,DBY_MSO,DBZ_MSO,RHO_DIPOLE,...,Z,VX,VY,VZ,VABS,D,COSALPHA,EXTREMA,ORBIT,LABEL
DATE,,,,,,,,,,,,,,,,,,,,,
2011-03-24 12:18:45,5804.272,7081.276,-2428.740,-44.922,16.665,4.784,4.239,6.375,3.369,9608.230719,...,5.855501e+06,-48.713393,-27.442143,2.227680,55.955593,4.894640e+07,0.633532,0,2,0
2011-03-24 12:18:46,5803.633,7080.696,-2427.421,-37.371,8.136,7.607,2.316,3.672,3.443,9607.017439,...,5.855501e+06,-48.713393,-27.442143,2.227663,55.955593,4.894640e+07,0.633532,0,2,0
2011-03-24 12:18:47,5802.993,7080.117,-2426.101,-34.104,4.656,14.430,1.949,0.970,1.545,9605.804096,...,5.855501e+06,-48.713393,-27.442143,2.227660,55.955593,4.894640e+07,0.633532,0,2,1
2011-03-24 12:18:48,5802.353,7079.538,-2424.782,-37.385,-0.449,23.100,1.797,3.131,4.607,9604.591160,...,5.855501e+06,-48.713393,-27.442143,2.227669,55.955593,4.894640e+07,0.633532,0,2,1
2011-03-24 12:18:49,5801.714,7078.958,-2423.462,-31.174,2.786,35.948,3.817,7.081,2.152,9603.377895,...,5.855501e+06,-48.713393,-27.442143,2.227612,55.955591,4.894640e+07,0.633532,0,2,1


Save the final training dataframe

In [12]:
df_train.to_csv(os.path.join(MERGED_DIR, "df_train.csv"))

Generate descriptive overall statistics

In [13]:
df_train_description = df_train.describe()
df_train_description.to_excel(os.path.join(MERGED_DIR, "df_train_total_description.xlsx"))
df_train_description

,X_MSO,Y_MSO,Z_MSO,BX_MSO,BY_MSO,BZ_MSO,DBX_MSO,DBY_MSO,DBZ_MSO,RHO_DIPOLE,...,Z,VX,VY,VZ,VABS,D,COSALPHA,EXTREMA,ORBIT,LABEL
count,1.023425e+08,1.023425e+08,1.023425e+08,1.023425e+08,1.023425e+08,1.023425e+08,1.023425e+08,1.023425e+08,1.023425e+08,1.023425e+08,...,1.023425e+08,1.023425e+08,1.023425e+08,1.023425e+08,1.023425e+08,1.023425e+08,1.023425e+08,1.023425e+08,1.023425e+08,1.023425e+08
mean,7.459365e+01,-1.579842e+02,-7.887372e+03,2.998895e-01,-2.245389e-01,-4.415617e+00,1.068106e+00,1.406365e+00,1.404429e+00,1.050174e+04,...,-1.420325e+06,2.152367e-01,2.984551e+00,2.241546e-01,4.744350e+01,5.904986e+07,-4.905159e-03,6.372717e-05,1.883164e+03,9.728163e-01
std,3.672142e+03,3.881640e+03,5.234757e+03,4.087478e+01,4.015112e+01,5.669018e+01,3.951213e+00,4.972515e+00,5.029860e+00,3.972189e+03,...,4.876415e+06,3.405189e+01,3.339439e+01,4.128471e+00,7.062540e+00,8.431919e+06,6.877777e-01,2.112260e-02,1.171942e+03,1.470978e+00
min,-1.074348e+04,-1.131478e+04,-1.662732e+04,-3.955740e+02,-9.754340e+03,-1.320918e+03,0.000000e+00,0.000000e+00,0.000000e+00,2.015986e+03,...,-7.654416e+06,-5.848620e+01,-4.661941e+01,-4.894014e+00,3.885789e+01,4.599997e+07,-1.000000e+00,-2.000000e+00,2.000000e+00,0.000000e+00
25%,-2.353351e+03,-2.792669e+03,-1.195803e+04,-1.765200e+01,-1.125300e+01,-8.293000e+00,1.820000e-01,2.180000e-01,2.400000e-01,7.753175e+03,...,-6.238016e+06,-3.199752e+01,-2.863935e+01,-3.773057e+00,4.059378e+01,5.051864e+07,-6.690732e-01,0.000000e+00,8.330000e+02,0.000000e+00
50%,-1.385020e+02,-7.687600e+01,-9.301120e+03,-4.290000e-01,5.400000e-02,5.820000e-01,5.200000e-01,7.020000e-01,7.140000e-01,1.136594e+04,...,-2.047131e+06,9.954904e+00,3.146155e+00,-5.571596e-01,4.594845e+01,6.028284e+07,-1.029296e-02,0.000000e+00,1.832000e+03,0.000000e+00
75%,2.490670e+03,2.533189e+03,-4.018852e+03,1.559300e+01,1.115500e+01,1.002700e+01,1.295000e+00,1.779000e+00,1.755000e+00,1.302481e+04,...,3.388680e+06,3.241589e+01,3.551466e+01,4.118227e+00,5.442669e+01,6.737373e+07,6.697104e-01,0.000000e+00,2.838000e+03,2.000000e+00
max,1.143029e+04,1.111089e+04,3.154425e+03,6.960477e+03,5.206690e+02,8.817431e+03,1.700265e+04,2.436953e+04,2.244895e+04,1.812033e+04,...,6.239501e+06,3.894385e+01,5.089473e+01,7.036499e+00,5.897771e+01,6.981782e+07,9.999954e-01,2.000000e+00,4.093000e+03,4.000000e+00


Generate descriptive label-wise statistics

In [14]:
df_train_description = df_train.groupby([LABEL_COL]).describe()
df_train_description.to_excel(os.path.join(MERGED_DIR, "df_train_label_description.xlsx"))
df_train_description

X_MSO                                                              \
            count         mean          std        min         25%        50%   
LABEL                                                                           
0      66215306.0  1293.785260  3401.199762  -9972.731 -1219.83600  1061.5170   
1       4116617.0 -1803.073803  3138.798580 -10142.408 -3688.73200 -2071.1740   
2      15102125.0 -2587.614541  3301.847868 -10743.482 -4761.90500 -2973.7920   
3       2394351.0 -2303.917850  3390.139020 -10611.648 -4873.47400 -2579.6060   
4      14514142.0 -1792.517474  2588.160964 -10289.062 -3807.19875 -1465.7615   

                                   Y_MSO              ... EXTREMA       \
              75%        max       count        mean  ...     75%  max   
LABEL                                                 ...                
0      3689.65275  11430.286  66215306.0 -359.347332  ...     0.0  2.0   
1        52.71500  10403.462   4116617.0   72.923658  ...     0.0  2.0   
2      -491.63400  10397.185  15102125.0  334.036583  ...     0.0  2.0   
3       512.36750   5104.713   2394351.0  251.150979  ...     0.0  1.0   
4       266.32100   4869.525  14514142.0  115.720813  ...     0.0  1.0   

            ORBIT                                                         \
            count         mean          std  min     25%     50%     75%   
LABEL                                                                      
0      66215306.0  1850.158500  1206.457428  2.0   745.0  1773.0  2877.0   
1       4116617.0  2033.333254  1115.099951  2.0  1096.0  2078.0  2897.0   
2      15102125.0  1920.799025  1113.688286  2.0  1007.0  1884.0  2813.0   
3       2394351.0  1964.880551  1130.236703  2.0  1008.0  1957.0  2866.0   
4      14514142.0  1938.505964  1083.136946  2.0  1075.0  1912.0  2779.0   

               
          max  
LABEL          
0      4093.0  
1      4093.0  
2      4093.0  
3      4093.0  
4      4093.0  

[5 rows x 232 columns]